# ⚡ HHL Codelab: Solving Linear Systems Quantum Fast

## Harrow-Hassidim-Lloyd Algorithm for Ax = b

| Property | Value |
|----------|-------|
| **Algorithm** | HHL (Harrow-Hassidim-Lloyd) |
| **Difficulty** | 🔴 Advanced |
| **Time** | 90-120 minutes |
| **Prerequisites** | Module-11-HHL-Algorithm.md, QPE |
| **Qiskit Version** | 2.x |

---

## 🎯 Learning Objectives

By completing this codelab, you will:

1. ✅ Implement QPE for eigenvalue extraction
2. ✅ Build controlled rotations for eigenvalue inversion
3. ✅ Construct complete HHL circuit for 2×2 systems
4. ✅ Verify HHL output against classical solutions
5. ✅ Understand the practical limitations

---

## Section 1: Environment Setup

In [ ]:
# Core imports
import numpy as np
import matplotlib.pyplot as plt
from numpy import pi

# Qiskit imports
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.primitives import StatevectorSampler
from qiskit_ibm_runtime.fake_provider import FakeAlmadenV2
from qiskit.quantum_info import Operator, Statevector
from qiskit.visualization import plot_histogram, plot_bloch_multivector

# Linear algebra
from scipy.linalg import expm
# Initialize fake backend and sampler
fake_backend = FakeAlmadenV2()
sampler = StatevectorSampler()

# Version check
import qiskit
print(f"Qiskit version: {qiskit.__version__}")
assert int(qiskit.__version__.split('.')[0]) >= 1, "Requires Qiskit 1.x or 2.x"
plt.rcParams['font.size'] = 12
# Configure plotting

plt.rcParams['figure.figsize'] = (10, 6)
print("✅ All imports successful!")

plt.rcParams['font.size'] = 12

---

## Section 2: Theory Recap

### The Problem

Given Hermitian matrix $A$ and vector $\mathbf{b}$, find $\mathbf{x}$ such that:
$$A\mathbf{x} = \mathbf{b}$$

### HHL Algorithm Steps

1. **Prepare** $|b\rangle$ - encode input vector
2. **QPE** - extract eigenvalues $\lambda_j$ into clock register
3. **Controlled Rotation** - encode $1/\lambda_j$ into ancilla amplitude
4. **Inverse QPE** - uncompute clock register
5. **Measure** ancilla - post-select on $|1\rangle$ for success

### Key Formula

If $|b\rangle = \sum_j \beta_j |u_j\rangle$ (eigenbasis of $A$), then:
$$|x\rangle \propto \sum_j \frac{\beta_j}{\lambda_j} |u_j\rangle$$

---

## Section 3: Problem Setup - A Simple 2×2 System

In [ ]:
# Define our test system: Ax = b
#
# We use a simple 2×2 Hermitian matrix with known eigenvalues
# This allows us to implement a tractable HHL circuit

# Matrix A - chosen for convenient eigenvalues
# A = [[1.5, 0.5], [0.5, 1.5]] has eigenvalues λ₁=1, λ₂=2
A = np.array([
    [1.5, 0.5],
    [0.5, 1.5]
])

# Input vector b
b = np.array([1.0, 0.0])  # Simple: just |0⟩ state

# Verify A is Hermitian
is_hermitian = np.allclose(A, A.conj().T)
print(f"Matrix A:")
print(A)
print(f"\nIs Hermitian: {is_hermitian}")

# Compute eigenvalues and eigenvectors
eigenvalues, eigenvectors = np.linalg.eigh(A)
print(f"\nEigenvalues: {eigenvalues}")
print(f"Condition number κ = {max(eigenvalues)/min(eigenvalues):.2f}")

# Classical solution
x_classical = np.linalg.solve(A, b)
print(f"\nClassical solution x = {x_classical}")
print(f"Normalized: x/||x|| = {x_classical / np.linalg.norm(x_classical)}")

# Verify: Ax should equal b
print(f"\nVerification A·x = {A @ x_classical}")
print(f"Original b = {b}")

---

## Section 4: Hamiltonian Simulation - Creating U = e^{iAt}

In [ ]:
def create_hamiltonian_evolution(A: np.ndarray, t: float) -> np.ndarray:
    """
    Create unitary U = e^{iAt} for Hamiltonian simulation.
    
    In QPE, we use U = e^{2πiA} scaled so eigenvalues fit in [0,1).
    
    Parameters:
        A: Hermitian matrix
        t: Evolution time
    
    Returns:
        Unitary matrix e^{iAt}
    """
    U = expm(1j * A * t)
    return U

# For QPE, we want eigenvalues to appear as phases
# U|u_j⟩ = e^{iλ_j t}|u_j⟩
# We choose t such that phases are distinguishable

# With eigenvalues λ=1 and λ=2, choose t=π
# This gives phases e^{iπ} = -1 and e^{2iπ} = 1
t = np.pi
U = create_hamiltonian_evolution(A, t)

print("Unitary U = e^{iAπ}:")
print(np.round(U, 4))

# Verify U is unitary
is_unitary = np.allclose(U @ U.conj().T, np.eye(2))
print(f"\nIs unitary: {is_unitary}")

# Check eigenvalues of U
U_eigenvalues = np.linalg.eigvals(U)
print(f"\nU eigenvalues (should be e^{{iλπ}}):")
for i, eig in enumerate(U_eigenvalues):
    phase = np.angle(eig) / np.pi
    print(f"  λ_{i+1} = {eigenvalues[i]:.1f} → e^{{i·{eigenvalues[i]:.1f}·π}} = {eig:.4f} (phase = {phase:.2f}π)")

---

## Section 5: Quantum Phase Estimation for HHL

In [ ]:
def qpe_for_hhl(
    qc: QuantumCircuit,
    clock_qubits: list,
    data_qubit: int,
    U: np.ndarray
) -> None:
    """
    Quantum Phase Estimation to extract eigenvalues.
    
    For HHL, this maps:
    |0⟩^⊗n |b⟩ → Σ_j β_j |λ̃_j⟩ |u_j⟩
    
    Parameters:
        qc: QuantumCircuit to modify
        clock_qubits: List of clock register qubit indices
        data_qubit: Index of data qubit
        U: Unitary to phase-estimate
    """
    n_clock = len(clock_qubits)
    
    # Step 1: Hadamard on clock qubits
    for q in clock_qubits:
        qc.h(q)
    
    # Step 2: Controlled-U^{2^k} operations
    for k, ctrl_qubit in enumerate(clock_qubits):
        # Apply U^{2^k} controlled by clock qubit k
        power = 2 ** k
        U_power = np.linalg.matrix_power(U, power)
        
        # Convert to controlled gate
        controlled_U = np.eye(4, dtype=complex)
        controlled_U[2:, 2:] = U_power
        
        qc.unitary(controlled_U, [ctrl_qubit, data_qubit], label=f'CU^{power}')
    
    # Step 3: Inverse QFT on clock register
    for i in range(n_clock // 2):
        qc.swap(clock_qubits[i], clock_qubits[n_clock - 1 - i])
    
    for i in range(n_clock):
        for j in range(i):
            angle = -np.pi / (2 ** (i - j))
            qc.cp(angle, clock_qubits[j], clock_qubits[i])
        qc.h(clock_qubits[i])

def inverse_qpe(
    qc: QuantumCircuit,
    clock_qubits: list,
    data_qubit: int,
    U: np.ndarray
) -> None:
    """
    Inverse QPE to uncompute eigenvalue register.
    """
    n_clock = len(clock_qubits)
    
    # Inverse of QFT^{-1} is QFT
    for i in range(n_clock - 1, -1, -1):
        qc.h(clock_qubits[i])
        for j in range(i - 1, -1, -1):
            angle = np.pi / (2 ** (i - j))
            qc.cp(angle, clock_qubits[j], clock_qubits[i])
    
    for i in range(n_clock // 2):
        qc.swap(clock_qubits[i], clock_qubits[n_clock - 1 - i])
    
    # Inverse of controlled-U operations
    for k in range(len(clock_qubits) - 1, -1, -1):
        ctrl_qubit = clock_qubits[k]
        power = 2 ** k
        U_power = np.linalg.matrix_power(U, power)
        U_power_dag = U_power.conj().T  # Inverse
        
        controlled_U_dag = np.eye(4, dtype=complex)
        controlled_U_dag[2:, 2:] = U_power_dag
        
        qc.unitary(controlled_U_dag, [ctrl_qubit, data_qubit], label=f'CU^{power}†')
    
    # Inverse Hadamards
    for q in clock_qubits:
        qc.h(q)

# Test QPE
print("Testing QPE for eigenvalue extraction...")

test_qc = QuantumCircuit(3)  # 2 clock + 1 data
# Prepare data qubit in eigenstate (eigenvector of A)
# First eigenvector is [1/√2, -1/√2] for our A
test_qc.h(2)  # |+⟩ is close to eigenvector for demo

qpe_for_hhl(test_qc, [0, 1], 2, U)
test_qc.measure_all()

print("\nQPE Test Circuit:")
print(test_qc.draw(output='text', fold=80))

---

## Section 5.5: State Evolution Analysis

### 5.5.1 The "Eigenvalue Inversion via Controlled Rotation" Rule

**The Key Identity**: For $|b\rangle = \sum_j \beta_j |u_j\rangle$ in eigenbasis of $A$:

$$|x\rangle = A^{-1}|b\rangle = \sum_j \frac{\beta_j}{\lambda_j} |u_j\rangle$$

**The Rule**: "Extract eigenvalues with QPE, encode their inverses as amplitudes via controlled rotation, then uncompute to leave only the solution."

**From the Lecture**:
> "The basic idea is can I find its eigenvalues? And then once I do that, I will just invert those eigenvalues, multiply it with the given vector b to find the solution vector. And what is the subroutine that allows me to find eigenvalues? We have already discussed that—that's quantum phase estimation."

**Why This Works**:

1. **Spectral decomposition**: Any Hermitian $A = \sum_j \lambda_j |u_j\rangle\langle u_j|$
2. **Inverse in eigenbasis**: $A^{-1} = \sum_j \frac{1}{\lambda_j} |u_j\rangle\langle u_j|$
3. **QPE extracts λ**: Maps $|u_j\rangle|0\rangle \to |u_j\rangle|\tilde{\lambda}_j\rangle$
4. **Controlled rotation encodes 1/λ**: $|\lambda_j\rangle|0\rangle \to |\lambda_j\rangle\left(\sqrt{1-\frac{C^2}{\lambda_j^2}}|0\rangle + \frac{C}{\lambda_j}|1\rangle\right)$
5. **Post-selection collapses to solution**: Measuring ancilla in $|1\rangle$ projects onto $\propto \sum_j \frac{\beta_j}{\lambda_j}|u_j\rangle$

### 5.5.2 State Evolution at Each Stage

| Stage | State | Physical Meaning |
|-------|-------|------------------|
| **Input** | $\|b\rangle\|0\rangle^{\otimes n}\|0\rangle$ | Load input vector, initialize clock and ancilla |
| **After QPE** | $\sum_j \beta_j \|u_j\rangle\|\tilde{\lambda}_j\rangle\|0\rangle$ | Eigenvalues encoded in clock register |
| **After Rotation** | $\sum_j \beta_j \|u_j\rangle\|\tilde{\lambda}_j\rangle\left(\sqrt{1-C^2/\lambda_j^2}\|0\rangle + \frac{C}{\lambda_j}\|1\rangle\right)$ | Inverse eigenvalues encoded as amplitudes |
| **After QPE⁻¹** | $\sum_j \beta_j \|u_j\rangle\|0\rangle^{\otimes n}\left(\sqrt{1-C^2/\lambda_j^2}\|0\rangle + \frac{C}{\lambda_j}\|1\rangle\right)$ | Clock uncomputed, solution in data register |
| **Post-select \|1⟩** | $\propto \sum_j \frac{\beta_j}{\lambda_j}\|u_j\rangle\|0\rangle^{\otimes n}\|1\rangle$ | **Solution**: $\|x\rangle = A^{-1}\|b\rangle$ |

**The Critical Insight** (from lecture):
> "Controlled on the value of lambda, perform a rotation of the ancilla. The amount in state |1⟩ is proportional exactly to one over lambda... If I do a measurement and I observe state |1⟩, I will collapse my system only onto this particular combination where I've produced this map."

In [ ]:
def trace_hhl_evolution(A: np.ndarray, b: np.ndarray, C: float = 0.5) -> None:
    """
    Trace the state evolution through HHL algorithm step by step.
    
    Demonstrates the "Eigenvalue Inversion via Controlled Rotation" rule.
    
    Parameters:
        A: Hermitian matrix
        b: Input vector
        C: Normalization constant for rotation (C ≤ λ_min)
    """
    print("=" * 70)
    print("HHL STATE EVOLUTION TRACE")
    print("=" * 70)
    
    # Eigendecomposition of A
    eigenvalues, eigenvectors = np.linalg.eigh(A)
    print(f"\n📐 Matrix A Eigendecomposition:")
    print(f"   A = Σⱼ λⱼ |uⱼ⟩⟨uⱼ|")
    for j, (lam, u) in enumerate(zip(eigenvalues, eigenvectors.T)):
        print(f"   λ_{j+1} = {lam:.4f}, |u_{j+1}⟩ = {u}")
    
    # Expand b in eigenbasis
    b_norm = b / np.linalg.norm(b)
    betas = eigenvectors.T @ b_norm  # Coefficients in eigenbasis
    
    print(f"\n📥 STAGE 1: Input State |b⟩|0⟩ⁿ|0⟩")
    print(f"   |b⟩ = {b_norm}")
    print(f"   In eigenbasis: |b⟩ = Σⱼ βⱼ|uⱼ⟩")
    for j, beta in enumerate(betas):
        print(f"      β_{j+1} = {beta:.4f} (contribution from |u_{j+1}⟩)")
    
    print(f"\n📊 STAGE 2: After QPE → Eigenvalue Extraction")
    print(f"   |ψ₂⟩ = Σⱼ βⱼ |uⱼ⟩|λ̃ⱼ⟩|0⟩")
    print(f"   Each eigenvector gets tagged with its eigenvalue:")
    for j, (beta, lam) in enumerate(zip(betas, eigenvalues)):
        print(f"      {beta:.4f}|u_{j+1}⟩|{lam:.2f}⟩|0⟩")
    
    print(f"\n🔄 STAGE 3: After Controlled Rotation → Eigenvalue Inversion")
    print(f"   For each λⱼ: rotate ancilla by θⱼ = 2·arcsin(C/λⱼ)")
    print(f"   Normalization constant C = {C}")
    
    for j, (beta, lam) in enumerate(zip(betas, eigenvalues)):
        theta = 2 * np.arcsin(min(C / lam, 1.0))
        amp_0 = np.sqrt(1 - (C/lam)**2)
        amp_1 = C / lam
        print(f"\n   λ_{j+1} = {lam:.2f}:")
        print(f"      θ_{j+1} = 2·arcsin({C}/{lam:.2f}) = {np.degrees(theta):.1f}°")
        print(f"      |λ_{j+1}⟩|0⟩ → |λ_{j+1}⟩({amp_0:.4f}|0⟩ + {amp_1:.4f}|1⟩)")
        print(f"      Note: amplitude on |1⟩ = C/λ = {C}/{lam:.2f} = {amp_1:.4f}")
    
    print(f"\n⏪ STAGE 4: After Inverse QPE → Clock Uncompute")
    print(f"   Eigenvalue register returns to |0⟩ⁿ")
    print(f"   But ancilla still carries the 1/λ information!")
    
    print(f"\n✅ STAGE 5: Post-Selection on Ancilla |1⟩")
    print(f"   Measuring ancilla = |1⟩ collapses to:")
    
    # Calculate post-selected state
    solution_amplitudes = []
    for j, (beta, lam) in enumerate(zip(betas, eigenvalues)):
        amp = beta * (C / lam)  # This is the post-selected amplitude
        solution_amplitudes.append(amp)
        print(f"      β_{j+1}/λ_{j+1} · C = {beta:.4f}/{lam:.2f} · {C} = {amp:.4f}")
    
    # Normalize
    solution_amplitudes = np.array(solution_amplitudes)
    norm = np.linalg.norm(solution_amplitudes)
    solution_normalized = solution_amplitudes / norm
    
    print(f"\n   Unnormalized: |x⟩ ∝ Σⱼ (βⱼ/λⱼ)|uⱼ⟩")
    print(f"   Normalized solution in eigenbasis: {solution_normalized}")
    
    # Convert back to computational basis
    x_quantum = eigenvectors @ solution_normalized
    print(f"\n   Solution in computational basis: |x⟩ = {x_quantum}")
    
    # Verify against classical
    x_classical = np.linalg.solve(A, b_norm)
    x_classical_norm = x_classical / np.linalg.norm(x_classical)
    print(f"\n🎯 VERIFICATION:")
    print(f"   Classical A⁻¹|b⟩ (normalized): {x_classical_norm}")
    print(f"   Match: {np.allclose(np.abs(x_quantum), np.abs(x_classical_norm))}")
    
    # Success probability
    success_prob = norm**2
    print(f"\n📈 Success Probability (P(ancilla=|1⟩)):")
    print(f"   P_success = ||Σⱼ (C·βⱼ/λⱼ)|uⱼ⟩||² = {success_prob:.4f} = {success_prob*100:.1f}%")
    
    return x_quantum

# Run the trace
print("Tracing HHL evolution for our 2×2 system...\n")
x_traced = trace_hhl_evolution(A, b, C=0.5)

### 5.5.3 Understanding the Controlled Rotation

The heart of HHL is encoding $1/\lambda$ as an amplitude. The rotation angle is:
$$\theta_j = 2\arcsin\left(\frac{C}{\lambda_j}\right)$$

This transforms the ancilla from $|0\rangle$ to:
$$R_Y(\theta_j)|0\rangle = \sqrt{1 - \frac{C^2}{\lambda_j^2}}|0\rangle + \frac{C}{\lambda_j}|1\rangle$$

**Why arcsin?** Because $\sin(\theta/2) = C/\lambda$, so the amplitude on $|1\rangle$ is exactly $C/\lambda$!

In [ ]:
def verify_rotation_angle(C: float = 0.5):
    """
    Verify the controlled rotation angle formula θ = 2·arcsin(C/λ).
    
    Demonstrates why this specific angle encodes 1/λ as amplitude.
    """
    print("=" * 60)
    print("CONTROLLED ROTATION VERIFICATION")
    print("=" * 60)
    
    print(f"\nNormalization constant C = {C}")
    print(f"Requirement: C ≤ λ_min (so C/λ ≤ 1 for valid arcsin)")
    
    eigenvalues = [1, 2, 3, 4, 5]  # Test eigenvalues
    
    print(f"\n{'λ':<6} {'θ = 2arcsin(C/λ)':<20} {'sin(θ/2)':<12} {'Amp on |1⟩':<12} {'= C/λ?':<10}")
    print("-" * 60)
    
    for lam in eigenvalues:
        if C <= lam:  # Valid range
            theta = 2 * np.arcsin(C / lam)
            sin_half = np.sin(theta / 2)
            amp_1 = C / lam
            match = "✓ Yes" if np.isclose(sin_half, amp_1) else "✗ No"
            print(f"{lam:<6.1f} {np.degrees(theta):<20.2f}° {sin_half:<12.4f} {amp_1:<12.4f} {match:<10}")
    
    print("\n🔑 KEY INSIGHT:")
    print("   The R_Y(θ) gate creates:")
    print("   |0⟩ → cos(θ/2)|0⟩ + sin(θ/2)|1⟩")
    print("   Setting θ = 2·arcsin(C/λ) makes sin(θ/2) = C/λ exactly!")
    print("\n   So measuring |1⟩ gives amplitude proportional to 1/λ.")
    
    # Visualize the rotation
    fig, ax = plt.subplots(1, 1, figsize=(10, 5))
    
    lam_range = np.linspace(C + 0.01, 5, 100)
    theta_range = 2 * np.arcsin(C / lam_range)
    amp_1_range = C / lam_range
    
    ax.plot(lam_range, np.degrees(theta_range), 'b-', linewidth=2, label='Rotation angle θ')
    ax.plot(lam_range, amp_1_range * 90, 'r--', linewidth=2, label='Amplitude C/λ (×90 for visibility)')
    ax.axhline(y=90, color='gray', linestyle=':', alpha=0.5, label='θ = 90° (full rotation)')
    ax.axvline(x=C, color='orange', linestyle=':', alpha=0.5, label=f'λ = C = {C}')
    
    ax.set_xlabel('Eigenvalue λ', fontsize=12)
    ax.set_ylabel('Rotation Angle θ (degrees)', fontsize=12)
    ax.set_title('HHL Rotation Angle vs Eigenvalue', fontsize=14)
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_xlim([C, 5])
    
    plt.tight_layout()
    plt.show()
    
    print("\n📊 From the plot:")
    print(f"   - Small λ (close to C={C}) → Large rotation (close to 90°) → Large 1/λ")
    print(f"   - Large λ → Small rotation → Small 1/λ")
    print(f"   - At λ = C: θ = 90°, amplitude = 1 (maximum)")

verify_rotation_angle(C=0.5)

### 5.5.4 Interactive: Explore Success Probability vs Condition Number

The **condition number** $\kappa = \lambda_{max}/\lambda_{min}$ dramatically affects HHL:
- Success probability $\propto 1/\kappa^2$
- More repetitions needed for ill-conditioned systems

In [ ]:
def explore_condition_number_effect(kappa_values: list = None):
    """
    Explore how condition number affects HHL success probability.
    
    From the lecture:
    "If the ratio of smallest and largest eigenvalues is large, that is κ is a large 
    number, then the number of steps required to actually find the solution also 
    becomes larger and larger."
    
    Parameters:
        kappa_values: List of condition numbers to test
    """
    if kappa_values is None:
        kappa_values = [1.5, 2, 3, 5, 10, 20, 50, 100]
    
    print("=" * 70)
    print("CONDITION NUMBER EFFECT ON HHL SUCCESS PROBABILITY")
    print("=" * 70)
    
    # For equal superposition b, success probability ∝ ||A⁻¹b||²
    # For well-conditioned: P_success ≈ 1
    # For ill-conditioned: P_success ≈ 1/κ²
    
    results = []
    
    for kappa in kappa_values:
        # Create diagonal matrix with eigenvalues 1 and kappa
        # This gives condition number exactly kappa
        A_test = np.diag([1.0, kappa])
        
        # Input b = |+⟩ ∝ [1, 1] (equal in eigenbasis)
        b_test = np.array([1, 1]) / np.sqrt(2)
        
        # Expand in eigenbasis (eigenvectors are computational basis for diagonal A)
        betas = b_test
        
        # C must satisfy C ≤ λ_min = 1, so set C = 0.5
        C = 0.5
        
        # Success probability: ||Σ (C·βⱼ/λⱼ)|uⱼ⟩||²
        amplitudes = [C * beta / lam for beta, lam in zip(betas, [1.0, kappa])]
        p_success = sum(amp**2 for amp in amplitudes)
        
        # Compare to well-conditioned case (κ=1)
        p_ideal = C**2  # When all eigenvalues are 1
        
        results.append({
            'kappa': kappa,
            'p_success': p_success,
            'ratio': p_success / p_ideal,
            'shots_needed': int(1 / p_success) if p_success > 0 else float('inf')
        })
    
    print(f"\n{'κ':<10} {'P_success':<15} {'vs κ=1':<15} {'Shots needed':<15}")
    print("-" * 55)
    
    for r in results:
        print(f"{r['kappa']:<10.1f} {r['p_success']:<15.4f} {r['ratio']:<15.2%} {r['shots_needed']:<15}")
    
    # Visualization
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    kappas = [r['kappa'] for r in results]
    p_successes = [r['p_success'] for r in results]
    shots_needed = [r['shots_needed'] for r in results]
    
    # Plot 1: Success probability
    ax1 = axes[0]
    ax1.loglog(kappas, p_successes, 'bo-', markersize=10, linewidth=2, label='HHL P_success')
    # Theoretical: P ∝ 1/κ² for large κ
    kappa_fit = np.array(kappas)
    p_theory = p_successes[0] * (kappas[0]**2) / (kappa_fit**2)
    ax1.loglog(kappas, p_theory, 'r--', linewidth=2, alpha=0.7, label='O(1/κ²) scaling')
    ax1.set_xlabel('Condition Number κ', fontsize=12)
    ax1.set_ylabel('Success Probability', fontsize=12)
    ax1.set_title('HHL Success vs Condition Number', fontsize=14)
    ax1.legend()
    ax1.grid(True, which='both', alpha=0.3)
    
    # Plot 2: Shots needed
    ax2 = axes[1]
    ax2.loglog(kappas, shots_needed, 'go-', markersize=10, linewidth=2, label='Shots needed')
    ax2.loglog(kappas, [shots_needed[0] * (k/kappas[0])**2 for k in kappas], 
               'r--', linewidth=2, alpha=0.7, label='O(κ²) scaling')
    ax2.set_xlabel('Condition Number κ', fontsize=12)
    ax2.set_ylabel('Expected Shots for 1 Success', fontsize=12)
    ax2.set_title('HHL Resource Cost vs Condition Number', fontsize=14)
    ax2.legend()
    ax2.grid(True, which='both', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\n🔑 KEY INSIGHTS from Lecture:")
    print("   1. HHL complexity scales as O(κ²) - quadratic in condition number")
    print("   2. For κ=100: need ~10,000× more measurements than κ=1!")
    print("   3. This is why HHL's 'exponential speedup' has asterisks")
    print("   4. Real-world matrices often have κ >> 1, limiting practical advantage")

explore_condition_number_effect()

### 5.5.5 The Input/Output Problem: Why HHL's Speedup Has Caveats

From the lecture:
> "If you just naively start loading the states B—that is, the vector b was given to you as a classical vector b₁, b₂, ..., bₙ—and now I need to encode this in states of some qubit lines... this naive way of reading the states will take at least n steps. And that just reading process would then overwhelm this advantage."

**The Three Caveats**:
1. **Input Problem**: Loading classical $\mathbf{b}$ into $|b\rangle$ requires $O(n)$ operations
2. **Output Problem**: Reading all components of $|x\rangle$ requires $O(n)$ measurements  
3. **Condition Number**: Success probability $\propto 1/\kappa^2$

In [ ]:
def demonstrate_hhl_caveats():
    """
    Demonstrate the input/output problem and when HHL provides advantage.
    
    From the lecture:
    "One more thing to keep in mind is the output side. If I want to read the data in 
    the solution... we cannot read the state vectors. We can only perform measurements 
    and extract global properties like expectation values."
    """
    print("=" * 70)
    print("HHL CAVEATS: WHEN DOES THE SPEEDUP EXIST?")
    print("=" * 70)
    
    print("\n📥 THE INPUT PROBLEM")
    print("-" * 50)
    
    n_values = [2, 4, 8, 16, 32, 64, 128, 256, 1024]
    print(f"{'System size n':<15} {'HHL O(log n)':<15} {'Classical load O(n)':<20}")
    print("-" * 50)
    for n in n_values:
        hhl_cost = int(np.ceil(np.log2(n)))
        load_cost = n
        speedup = "❌ Lost!" if load_cost > 10 * hhl_cost else "✓ Possible"
        print(f"{n:<15} {hhl_cost:<15} {load_cost:<20} {speedup}")
    
    print("\n   💡 Solution: qRAM (hypothetical) or quantum-native data")
    print("      If data is produced by another quantum algorithm, no loading needed!")
    
    print("\n" + "=" * 70)
    print("📤 THE OUTPUT PROBLEM")
    print("-" * 50)
    
    print("\n   HHL gives you |x⟩ = Σⱼ αⱼ|j⟩, NOT the classical vector x = (x₁, ..., xₙ)")
    print("\n   What you CAN extract efficiently:")
    print("      ✅ Expectation values: ⟨x|M|x⟩ for any observable M")
    print("      ✅ Samples from probability distribution |αⱼ|²")
    print("      ✅ Use |x⟩ as input to another quantum subroutine")
    
    print("\n   What you CANNOT extract efficiently:")
    print("      ❌ All n components of x individually")
    print("      ❌ The actual numerical values x₁, x₂, ..., xₙ")
    
    print("\n   📊 To extract all components with precision ε:")
    print("      Need O(n/ε²) measurements → O(n) complexity!")
    print("      This erases any quantum speedup.")
    
    print("\n" + "=" * 70)
    print("🎯 WHEN HHL PROVIDES ADVANTAGE")
    print("-" * 50)
    
    use_cases = [
        ("✅ Quantum ML subroutine", "Use |x⟩ inside larger quantum algorithm"),
        ("✅ Expectation values", "Need ⟨x|M|x⟩ not individual xᵢ"),
        ("✅ Quantum data", "Input comes from quantum sensor/simulation"),
        ("❌ Full solution readout", "Need all classical values x₁...xₙ"),
        ("❌ Dense matrices", "Sparsity s ~ n erases speedup"),
        ("❌ Ill-conditioned", "κ >> 1 requires O(κ²) repetitions"),
    ]
    
    for status, description in use_cases:
        print(f"   {status}: {description}")
    
    print("\n" + "=" * 70)
    print("🔬 DEQUANTIZATION: The Tang Result")
    print("-" * 50)
    print("""
   In 2018, Ewin Tang (as an 18-year-old!) showed that classical algorithms
   can solve certain problems thought to require HHL in polynomial time.
   
   Key insight: If you only need expectation values (not full state),
   and have "sample and query" access to data, classical algorithms
   can sometimes match quantum performance up to polynomial factors.
   
   This sparked the "dequantization" research program questioning
   whether exponential quantum ML speedups truly exist.
    """)
    
    print("   📚 Reference: Tang, 'A Quantum-Inspired Classical Algorithm for")
    print("      Recommendation Systems', STOC 2019")

demonstrate_hhl_caveats()

### 5.5.6 Summary: Why HHL Works (and When It Doesn't)

| Component | Formula | Role in Algorithm |
|-----------|---------|-------------------|
| **Input** | $\|b\rangle = \sum_j \beta_j \|u_j\rangle$ | Expand input in eigenbasis of $A$ |
| **QPE** | $\|u_j\rangle\|0\rangle \to \|u_j\rangle\|\tilde{\lambda}_j\rangle$ | Extract eigenvalues into clock register |
| **Rotation** | $R_Y(2\arcsin(C/\lambda_j))$ | Encode $1/\lambda_j$ as amplitude on ancilla $\|1\rangle$ |
| **Uncompute** | QPE$^{-1}$ | Disentangle clock register, leave solution in data |
| **Post-select** | Measure ancilla $= \|1\rangle$ | Collapse to $\|x\rangle \propto \sum_j (\beta_j/\lambda_j)\|u_j\rangle$ |

**The "Eigenvalue Inversion via Controlled Rotation" Rule**:
> "Use QPE to extract eigenvalues λⱼ. Apply controlled rotation θⱼ = 2·arcsin(C/λⱼ) to encode 1/λⱼ as amplitude. Uncompute QPE. Post-select on ancilla |1⟩ to obtain |x⟩ = A⁻¹|b⟩."

**Key Insights from the Lecture**:

1. **Why eigendecomposition?** $A^{-1} = \sum_j (1/\lambda_j)|u_j\rangle\langle u_j|$ is diagonal in eigenbasis
2. **Why controlled rotation?** Maps $\lambda_j \to 1/\lambda_j$ as amplitude (non-unitary via post-selection)
3. **Why uncompute?** Clock register must return to $|0\rangle^n$ to not corrupt solution
4. **Why O(log n)?** Only need O(log n) qubits to store n-dimensional system

**When HHL Provides Exponential Speedup**:
- ✅ Input is quantum-native (no classical loading)
- ✅ Output is expectation value (no full readout)
- ✅ Matrix is sparse (s << n)
- ✅ Condition number is small (κ ~ O(1))

**Complexity**: O(log(n) · s² · κ² / ε) vs classical O(n · s · κ · log(1/ε))

---

## Section 6: Controlled Rotation for Eigenvalue Inversion

In [ ]:
def eigenvalue_inversion_rotation(
    qc: QuantumCircuit,
    clock_qubits: list,
    ancilla: int,
    eigenvalues: list,
    C: float = 0.5
) -> None:
    """
    Apply controlled rotation to encode 1/λ in ancilla amplitude.
    
    For eigenvalue λ encoded in clock, rotate ancilla:
    |λ⟩|0⟩ → |λ⟩(√(1-C²/λ²)|0⟩ + C/λ|1⟩)
    
    Parameters:
        qc: QuantumCircuit
        clock_qubits: Clock register qubit indices
        ancilla: Ancilla qubit index
        eigenvalues: List of eigenvalues (for angle calculation)
        C: Normalization constant (C ≤ min eigenvalue)
    """
    n_clock = len(clock_qubits)
    
    # For a 2-qubit clock, we have 4 possible states: |00⟩, |01⟩, |10⟩, |11⟩
    # These encode different eigenvalues based on QPE output
    
    # For our example with λ=1, λ=2:
    # |01⟩ → λ=1, rotate by θ₁ = 2*arcsin(C/1)
    # |10⟩ → λ=2, rotate by θ₂ = 2*arcsin(C/2)
    
    # Map clock states to eigenvalues (simplified for 2-bit precision)
    # In practice, this mapping depends on how QPE encodes phases
    
    for state_int, lam in enumerate(eigenvalues):
        if lam > 0:  # Avoid division by zero
            # Rotation angle: θ = 2*arcsin(C/λ)
            angle = 2 * np.arcsin(min(C / lam, 1.0))  # Clamp to valid arcsin range
            
            # Create multi-controlled Ry gate
            # Control on specific clock state, target ancilla
            state_bits = format(state_int + 1, f'0{n_clock}b')  # +1 because QPE encodes 1,2 not 0,1
            
            # Add X gates for 0-controls (to make it controlled on specific bitstring)
            for i, bit in enumerate(reversed(state_bits)):
                if bit == '0':
                    qc.x(clock_qubits[i])
            
            # Multi-controlled Ry
            qc.mcry(angle, clock_qubits, ancilla)
            
            # Undo X gates
            for i, bit in enumerate(reversed(state_bits)):
                if bit == '0':
                    qc.x(clock_qubits[i])

# Simplified version for our specific problem
def simple_rotation_for_2x2(
    qc: QuantumCircuit,
    clock_qubits: list,
    ancilla: int,
    C: float = 0.5
) -> None:
    """
    Simplified rotation for 2×2 HHL with eigenvalues 1 and 2.
    
    Clock state |01⟩ → λ=1 → rotate by 2*arcsin(C)
    Clock state |10⟩ → λ=2 → rotate by 2*arcsin(C/2)
    """
    # For λ=1: full rotation
    theta_1 = 2 * np.arcsin(C)  # When C=0.5, θ=π/3
    # For λ=2: half rotation
    theta_2 = 2 * np.arcsin(C / 2)  # When C=0.5, θ=π/6
    
    # Controlled on |01⟩ (λ=1)
    qc.x(clock_qubits[1])  # Flip to make it |10⟩ control
    qc.mcry(theta_1, clock_qubits, ancilla)
    qc.x(clock_qubits[1])
    
    # Controlled on |10⟩ (λ=2)
    qc.x(clock_qubits[0])
    qc.mcry(theta_2, clock_qubits, ancilla)
    qc.x(clock_qubits[0])

print("Eigenvalue Inversion Angles:")
C = 0.5
for lam in [1, 2]:
    theta = 2 * np.arcsin(C / lam)
    amplitude = C / lam
    print(f"  λ={lam}: θ={theta:.4f} rad = {np.degrees(theta):.1f}°, amplitude C/λ = {amplitude:.3f}")

---

## Section 7: Complete HHL Circuit

In [ ]:
def create_hhl_circuit(
    A: np.ndarray,
    b: np.ndarray,
    n_clock: int = 2,
    C: float = 0.5
) -> QuantumCircuit:
    """
    Create complete HHL circuit for 2×2 system.
    
    Parameters:
        A: 2×2 Hermitian matrix
        b: Input vector (will be normalized)
        n_clock: Number of clock qubits for QPE precision
        C: Normalization constant for rotation
    
    Returns:
        Complete HHL QuantumCircuit
    """
    # Registers
    clock = QuantumRegister(n_clock, 'clock')
    data = QuantumRegister(1, 'data')
    ancilla = QuantumRegister(1, 'ancilla')
    c_ancilla = ClassicalRegister(1, 'c_anc')
    c_data = ClassicalRegister(1, 'c_data')
    
    qc = QuantumCircuit(clock, data, ancilla, c_ancilla, c_data)
    
    # Create unitary U = e^{iAt}
    t = np.pi
    U = expm(1j * A * t)
    
    # ===== Step 1: Prepare |b⟩ =====
    # For b = [1, 0], data qubit stays |0⟩
    # For general b, use state preparation
    b_norm = b / np.linalg.norm(b)
    if not np.allclose(b_norm, [1, 0]):
        # General state preparation
        theta = 2 * np.arccos(b_norm[0])
        qc.ry(theta, data[0])
    
    qc.barrier(label='Prep |b⟩')
    
    # ===== Step 2: QPE =====
    qpe_for_hhl(qc, list(range(n_clock)), n_clock, U)
    qc.barrier(label='QPE')
    
    # ===== Step 3: Controlled Rotation =====
    simple_rotation_for_2x2(qc, list(range(n_clock)), n_clock + 1, C)
    qc.barrier(label='Rotate')
    
    # ===== Step 4: Inverse QPE =====
    inverse_qpe(qc, list(range(n_clock)), n_clock, U)
    qc.barrier(label='QPE⁻¹')
    
    # ===== Step 5: Measurement =====
    qc.measure(ancilla, c_ancilla)
    qc.measure(data, c_data)
    
    return qc

# Create HHL circuit
hhl_circuit = create_hhl_circuit(A, b, n_clock=2, C=0.5)

print("Complete HHL Circuit:")
print(hhl_circuit.draw(output='text', fold=120))

print(f"\nCircuit Statistics:")
print(f"  Total qubits: {hhl_circuit.num_qubits}")
print(f"  Circuit depth: {hhl_circuit.depth()}")

---

## Section 8: Execute HHL and Analyze Results

In [ ]:
def run_hhl(circuit: QuantumCircuit, shots: int = 8192) -> dict:
    """
    Execute HHL circuit and process results.
    
    Returns:
        Dictionary with success rate and solution statistics
    """
    # Transpile and run with SamplerV2
    transpiled_qc = transpile(circuit, backend=fake_backend, optimization_level=3)
    job = sampler.run([transpiled_qc], shots=shots)
    result = job.result()
    
    # Extract counts from DataBin
    pub_result = result[0].data
    
    # Get the measurement data for each classical register
    # BitArray objects have get_counts() method for easy counting
    c_anc_data = pub_result.c_anc  # Ancilla measurements
    c_data_data = pub_result.c_data  # Data qubit measurements
    
    # Convert BitArray to integer arrays for easier processing
    # get_int_counts() returns a dictionary of integer values to counts
    anc_counts = c_anc_data.get_int_counts()
    data_counts = c_data_data.get_int_counts()
    
    # Combine measurements by iterating through shots
    counts = {}
    for shot_idx in range(shots):
        # Get bits for this shot using get_bitstrings()
        anc_bits = c_anc_data.get_bitstrings()[shot_idx]
        data_bits = c_data_data.get_bitstrings()[shot_idx]
        
        # Format: 'data_bit ancilla_bit'
        bitstring = f"{data_bits} {anc_bits}"
        counts[bitstring] = counts.get(bitstring, 0) + 1
    
    # Parse results
    success_counts = {}
    failure_counts = {}
    
    for bitstring, count in counts.items():
        parts = bitstring.split()
        data_bit, ancilla_bit = parts[0], parts[1]
        
        if ancilla_bit == '1':  # Success
            success_counts[data_bit] = success_counts.get(data_bit, 0) + count
        else:  # Failure
            failure_counts[data_bit] = failure_counts.get(data_bit, 0) + count
    
    total_success = sum(success_counts.values())
    total_failure = sum(failure_counts.values())
    success_rate = total_success / (total_success + total_failure) if (total_success + total_failure) > 0 else 0
    
    # Extract solution distribution from successful runs
    if total_success > 0:
        p0 = success_counts.get('0', 0) / total_success
        p1 = success_counts.get('1', 0) / total_success
        
        # Reconstructed solution (amplitudes)
        x_quantum = np.array([np.sqrt(p0), np.sqrt(p1)])
    else:
        x_quantum = np.array([0, 0])
        p0 = p1 = 0
    
    return {
        'counts': counts,
        'success_counts': success_counts,
        'failure_counts': failure_counts,
        'success_rate': success_rate,
        'x_quantum': x_quantum,
        'p0': p0,
        'p1': p1
    }

# Run HHL
print("Running HHL circuit...")
results = run_hhl(hhl_circuit, shots=8192)

print(f"\n📊 HHL Results:")
print(f"  Success rate (ancilla=|1⟩): {results['success_rate']*100:.1f}%")
print(f"  Successful measurements: {results['success_counts']}")

print(f"\n🔬 Solution Analysis:")
print(f"  P(|0⟩) = {results['p0']:.4f}")
print(f"  P(|1⟩) = {results['p1']:.4f}")
print(f"  Quantum solution (amplitudes): {results['x_quantum']}")

# Compare to classical
x_classical_norm = x_classical / np.linalg.norm(x_classical)
print(f"\n  Classical solution (normalized): {x_classical_norm}")

print(f"  Classical |α|²: {x_classical_norm[0]**2:.4f}")
print(f"  Classical |β|²: {x_classical_norm[1]**2:.4f}")

---

## Section 9: Visualization

In [ ]:
# Visualize results
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Plot 1: All measurement outcomes
ax1 = axes[0]
all_counts = results['counts']
labels = list(all_counts.keys())
values = list(all_counts.values())
# Simplified coloring: green if ancilla bit (first bit in measurement) is '1'
colors = ['green' if '1' in l else 'red' for l in labels]
ax1.bar(labels, values, color=colors)
ax1.set_xlabel('Measurement Outcome')
ax1.set_ylabel('Counts')
ax1.set_title('All HHL Measurements')
ax1.tick_params(axis='x', rotation=45)

# Plot 2: Success vs Failure
ax2 = axes[1]
success_total = sum(results['success_counts'].values())
failure_total = sum(results['failure_counts'].values())
ax2.pie([success_total, failure_total], 
        labels=[f'Success\n({success_total})', f'Failure\n({failure_total})'],
        colors=['green', 'lightcoral'],
        autopct='%1.1f%%')
ax2.set_title('HHL Success Rate\n(Ancilla = |1⟩)')

# Plot 3: Solution comparison
ax3 = axes[2]
x = np.arange(2)
width = 0.35
quantum_probs = [results['p0'], results['p1']]
classical_probs = [x_classical_norm[0]**2, x_classical_norm[1]**2]

bars1 = ax3.bar(x - width/2, quantum_probs, width, label='HHL (Quantum)', color='steelblue')
bars2 = ax3.bar(x + width/2, classical_probs, width, label='Classical', color='coral')
ax3.set_xlabel('Basis State')
ax3.set_ylabel('Probability')
ax3.set_title('Solution Comparison')
ax3.set_xticks(x)
ax3.set_xticklabels(['|0⟩', '|1⟩'])
ax3.legend()

plt.tight_layout()
plt.show()

# Calculate fidelity
fidelity = (np.sqrt(results['p0']) * x_classical_norm[0] + 
            np.sqrt(results['p1']) * np.abs(x_classical_norm[1]))**2
print(f"\n📈 Solution Fidelity: {fidelity:.4f}")

---

## Section 10: Condition Number Analysis

In [ ]:
def analyze_condition_number_effect():
    """
    Demonstrate how condition number affects HHL success rate.
    """
    # Create matrices with different condition numbers
    # A = [[a, c], [c, b]] has eigenvalues (a+b)/2 ± sqrt((a-b)²/4 + c²)
    
    test_matrices = []
    condition_numbers = []
    
    for ratio in [1.5, 2, 3, 5, 10]:
        # Create matrix with eigenvalues 1 and ratio
        # Eigenvalues: λ₁ = 1, λ₂ = ratio
        # For diagonal matrix: A = [[1, 0], [0, ratio]]
        A_test = np.array([[1, 0], [0, ratio]])
        
        eigvals = np.linalg.eigvalsh(A_test)
        kappa = max(eigvals) / min(eigvals)
        
        test_matrices.append(A_test)
        condition_numbers.append(kappa)
    
    print("Condition Number Analysis:")
    print("="*50)
    print(f"{'κ (condition #)':<20} {'Expected Success Rate':<25}")
    print("-"*50)
    
    # Theoretical success probability ∝ ||A^{-1}b||^2
    # For b = [1,0] and diagonal A, x = [1, 0], so success ≈ 1
    # But post-selection probability depends on C/λ_min vs C/λ_max
    
    for kappa in condition_numbers:
        # Simplified model: success ∝ (λ_min / λ_max)² = 1/κ²
        # In practice, it's more nuanced
        approx_success = 1 / kappa  # Rough approximation
        print(f"{kappa:<20.1f} {approx_success*100:<25.1f}%")
    
    return condition_numbers

kappas = analyze_condition_number_effect()

print("\n⚠️ Key Insight:")
print("  Higher condition number → Lower success probability")
print("  HHL complexity scales as O(κ²)")
print("  For κ=100, you need ~10,000× more shots!")

---

## Section 11: Trap Demonstrations

In [ ]:
print("🚨 TRAP 1: The Input Problem")
print("="*50)

n_values = [2, 4, 8, 16, 32, 64, 128]
print(f"{'n (matrix size)':<20} {'HHL (log n)':<15} {'Data Loading (n)':<20}")
print("-"*55)
for n in n_values:
    hhl_cost = np.log2(n)
    load_cost = n
    print(f"{n:<20} {hhl_cost:<15.1f} {load_cost:<20}")

print("\n⚠️ Lesson: Unless you have qRAM or quantum input,")
print("   data loading dominates and you lose the speedup!")

print("\n" + "="*50)
print("🚨 TRAP 2: The Output Problem")
print("="*50)

print("\nHHL gives you |x⟩, not x!")
print("\nWhat you CAN efficiently extract:")
print("  ✅ Expectation values ⟨x|M|x⟩")
print("  ✅ Samples from |x|² distribution")
print("  ✅ Use |x⟩ as input to another quantum algorithm")

print("\nWhat you CANNOT efficiently extract:")
print("  ❌ All n components of x")
print("  ❌ The actual numerical values x_1, x_2, ..., x_n")

# Demonstrate: extracting all components requires O(n) measurements
print(f"\nFor n={128} system:")
print(f"  To extract all components with precision ε:")
print(f"  Need O(n/ε²) = O({128}/ε²) measurements")
print(f"  This erases the quantum advantage!")

---

## Section 12: Exercises

### 🟢 Exercise 1: Different Input Vector

Modify the HHL circuit to solve Ax = b where b = [1/√2, 1/√2] (the |+⟩ state).

In [ ]:
# TODO: Create HHL circuit with b = [1/√2, 1/√2]
# Hint: The data qubit should start in |+⟩ state (apply H gate)

# YOUR CODE HERE

### 🟡 Exercise 2: Verify Post-Selection

Compare results with and without post-selection on the ancilla qubit.

In [ ]:
# TODO: Analyze what happens when ancilla = |0⟩ (failure case)
# Compare the data qubit distribution in success vs failure cases

# YOUR CODE HERE

### 🔴 Exercise 3: Expectation Value Extraction

Given the HHL solution |x⟩, compute ⟨x|Z|x⟩ (the expectation value of Z).

In [ ]:
# TODO: Compute ⟨x|Z|x⟩ from HHL measurements
# Hint: ⟨Z⟩ = P(|0⟩) - P(|1⟩)

# YOUR CODE HERE

---

## Section 13: Quick Checks ✅

In [ ]:
print("🧠 Quick Check Questions")
print("="*50)

questions = [
    {
        'q': "1. What is the purpose of the controlled rotation in HHL?",
        'a': "To encode 1/λ in the ancilla amplitude. The rotation angle θ = 2*arcsin(C/λ) puts amplitude C/λ on |1⟩."
    },
    {
        'q': "2. Why do we need inverse QPE in HHL?",
        'a': "To uncompute (disentangle) the eigenvalue register. Without it, the clock qubits remain entangled with the data, corrupting the solution."
    },
    {
        'q': "3. What does measuring ancilla = |0⟩ mean?",
        'a': "Failure! The controlled rotation didn't fully work. We discard this result and try again (post-selection)."
    },
    {
        'q': "4. Why doesn't HHL give exponential speedup for all linear systems?",
        'a': "Caveats: (1) Input loading takes O(n), (2) Output extraction takes O(n), (3) High κ kills speedup, (4) Dense matrices lose the advantage."
    }
]

for q_dict in questions:
    print(f"\n{q_dict['q']}")
    input("Press Enter to see answer...")
    print(f"Answer: {q_dict['a']}")

---

## Section 14: Summary and Next Steps

### Key Takeaways

1. **HHL Structure**: Prepare |b⟩ → QPE → Rotate → Inverse QPE → Measure

2. **Eigenvalue Inversion**: Controlled rotation encodes 1/λ as amplitude on ancilla |1⟩

3. **Post-Selection**: Success when ancilla = |1⟩; probability depends on ||A⁻¹|b⟩||²

4. **Critical Caveats**:
   - Input problem (qRAM needed)
   - Output problem (can't read full solution)
   - Condition number dependence O(κ²)
   - Sparsity requirement

### What You've Implemented

- ✅ Hamiltonian evolution U = e^{iAt}
- ✅ QPE for eigenvalue extraction
- ✅ Controlled rotation for inversion
- ✅ Complete HHL for 2×2 systems
- ✅ Results analysis and verification

### Next Steps

- **Module 12**: Quantum Neural Networks
- **Extension**: Implement HHL for larger systems using block-encoding
- **Application**: Use HHL as subroutine in quantum ML algorithms